# Purpose:
This script will create a dict of trips with a car bin assignment randomly assigned in proprtion to the Riyadh fleet bins

Inputs:
    -

In [28]:
import pandas as pd
import numpy as np
import cProfile
import time
import networkx as nx
import pandas as pd

import cPickle as pickle
import matplotlib.pyplot as plt
import matplotlib
from scipy import stats
from numpy import polyval
from collections import defaultdict
from joblib import Parallel, delayed
import multiprocessing
import sys
import warnings
import cPickle as pickle
from make_multiplex import *   # for read_metro(or streets) and clean_
from metro.utility import *
from fuel_calc import *

In [5]:
# Load the ITA results with Paths followed by ODs
df100 = pd.read_csv('data/DataFrame0_1_100pc_.csv')


In [21]:
j = pickle.load(open("data/mx_Riyadh_streets_TAZ_ODs_0_1.pkl", "rb"))
g, od = j.to_igraph()

In [22]:
paths2gid_map = { e.index : e['gid'] for e in g.es}

In [23]:
# Filters out flows less than one car/hour. Divides by 4 since the ODs are for a 4 hour period
df100_sorted = df100[df100['flow']/4>1][['flow','o_con','d_con', 'path']].sort_values(by=['flow'], ascending=False)


In [25]:
# The proportion of cars in Riyadh Fleet
Riyadh_fleet_prop = [0.0016624,
0.0003830,
0.0227833,
0.0440881,
0.0487076,
0.0481790,
0.0419814,
0.1098564,
0.0383885,
0.3886037,
0.1585104,
0.0416826,
0.0549435,
0.0002298]

probs = np.array(Riyadh_fleet_prop)
probs /= probs.sum()


1.0

Create Dictionary for all trips in Time 1 to fill details of path and Fuel consumption as attributes

In [26]:
import time
import ast
start = time.time()
rand_flow=[]
tripnumber = 0
Trips_time1_100_p1 = {}
factor = 1.5  
hours_of_period = 4
for ind in range(len(df100[df100['flow']/4>1])):
    pp=df100_sorted.iloc[ind]
    paths_parsed = ast.literal_eval(pp[3].replace('][', '],['))
    if len(paths_parsed[0]) == 0: continue
    paths_gid = []
    for p in range(4):
        paths_gid.append(map(paths2gid_map.get, paths_parsed[p][1:-1])) # converts igraph edges to nx street gid
    
    
    flow = pp[0]/4 # divided by 4 as a result of the output method [car/morning period]
    flow = flow*factor/hours_of_period # factored by 1.5 for peak hour and divided by 4 for hourly rate [car/hour]
    for rr in range(int(round(flow))):
        car_FEbin= np.random.choice(14, p = probs)
        Trips_time1_100_p1[tripnumber]={'o_con':pp[1],'d_con':pp[2],'path': paths_gid[0], 'car_FEbin':car_FEbin}
        tripnumber += 1
    if round(flow) == 0: break
print 'Time taken is', time.time() - start, 'sec'


Time taken is 373.240859985 sec


In [27]:
# Load riyadh edge congested times 
route_edges_am = pd.read_csv('data/riyadh_route_edges_am.txt', delimiter=' ')


In [38]:
# Load streets network with FC by bin as an attribute of each edge
with open("data/Riyadh_streets_FCcalculated_time1_targeted_reduction.pkl","rb") as i:
    streets_1 = pickle.load(i)


In [33]:
# Build Gid_2_edge dict
gid_2_edge = {}
for n1,n2 in streets_1.edges():

    gid_2_edge[streets_1[n1][n2]['gid']] = (n1, n2)

In [ ]:
# Parallelizing the Fuel consumption per trip calculation over 10 cores

In [29]:
No_trips = len(Trips_time1_100_p1)
Random_rank_all_trips_time1_100 = np.random.choice(No_trips, No_trips, replace=False)


In [41]:
array_trip=[]
div=int(len(Trips_time1_100_p1)/10.0)
app={}
k=1
for keys in Random_rank_all_trips_time1_100:
    app[keys]=Trips_time1_100_p1[keys]   
    if k%div==0:
        array_trip.append(app)
        app={}
    k=k+1

In [40]:
matched_Edges_1 = []
for e1,e2 in streets_1.edges():
    time1 = streets_1[e1][e2]['FC_bybin_time1']
    if len(time1) > 0: matched_Edges_1.append(streets_1[e1][e2]['gid'])


In [43]:
mat_edg1=defaultdict(int)
for k in matched_Edges_1:
    mat_edg1[k]=1


In [63]:
attrs = ['FC_tripd','CS_FC_time1','CS_FC_tripd' ]
def functrips(pos):
    kl=0
    start=time.time()
    totp=float(len(pos))
    for trip, details in pos.iteritems():
        kl=kl+1
        tm = 0 #[hr]
        dist = 0
        FC_bytrip = 0
        car_FEbin = details['car_FEbin']
        for gid in details['path']:
            e1,e2 = gid_2_edge[gid]
            if mat_edg1[gid]!=1: continue # skips edges without fuel for time 1. For comparison with SP fuel cons
            time_path = route_edges_am[route_edges_am['eid']==gid]['travel_time'].values[0] # for [min]
            tm += time_path # congested time in [min]
            dist += streets_1[e1][e2]['len_km'] # [km]
            FC_bytrip += streets_1[e1][e2]['FC_bybin_time1'][0][car_FEbin] # [us gal]
        details['triptime1_m'] = tm # [min]
        if dist == 0:
            for attr in attrs:
                details[attr] = 0
            continue
        details['FC_time1'] = FC_bytrip*3.78541 #[liter]
        details['FC_tripd'] = (FC_bytrip*3.78541)/dist # [l/km]
        CS_FC_time1 = Fuelconsump_Riyadhbin(0,tm*60,0,dist, [car_FEbin])
        details['CS_FC_time1'] = CS_FC_time1*3.78541 #[liter]
        details['CS_FC_tripd'] = (CS_FC_time1*3.78541)/dist # [l/km]

        if kl == int(totp*0.01): print '1%  done in', time.time() - start, 'sec'
        if kl == int(totp*0.1): print '10%  done in', time.time() - start, 'sec'
        if kl == int(totp*0.3): print '30%  done in', time.time() - start, 'sec'
        if kl == int(totp*0.6): print '60%  done in', time.time() - start, 'sec'
        if kl == int(totp*0.9): print '90%  done in', time.time() - start, 'sec'


        
    return pos


In [ ]:
pl2=Parallel(n_jobs=10)(delayed(functrips)(il) for il in array_trip)

1%  done in 37.9037060738 sec
1%  done in 37.3717279434 sec
1%  done in 35.4229888916 sec
1%  done in 39.0108659267 sec
1%  done in 36.1518950462 sec
1%  done in 44.8389930725 sec
1%  done in 47.6067779064 sec
1%  done in 50.3270530701 sec
10%  done in 441.595355988 sec
10%  done in 418.067780018 sec
10%  done in 447.092484951 sec
10%  done in 424.754855871 sec
10%  done in 429.368075848 sec
10%  done in 498.370569944 sec
10%  done in 508.028415918 sec
10%  done in 455.457504034 sec
10%  done in 517.989043951 sec
10%  done in 538.119520903 sec
30%  done in 1382.90139389 sec
30%  done in 1361.05912495 sec
30%  done in 1523.47536206 sec
30%  done in 1431.591856 sec
30%  done in 1543.82080197 sec
30%  done in 1606.54575801 sec
30%  done in 1597.9485631 sec
30%  done in 1587.83177996 sec
30%  done in 1564.99948692 sec
60%  done in 2828.992208 sec
60%  done in 2783.35126781 sec
60%  done in 2824.85623384 sec
60%  done in 3183.5289309 sec
60%  done in 3179.35184002 sec
60%  done in 3201.0762

In [67]:
# collect the results of pl, corrected to reciprocate and save the results to the original trip dict
trip_tottime_m2 =[]
trip_dist_km =[]

attribs = ['triptime1_m','FC_time1','FC_tripd','CS_FC_time1','CS_FC_tripd']
for i, array in enumerate(pl2):
    for trip,details in array.iteritems():    
        for attr in attribs:
            
            if details['FC_time1'] == 0:
                Trips_time1_100_p1[trip][attr] = 0  
                Trips_time1_100_p1[trip]['tripLen_km'] = 0
            else: Trips_time1_100_p1[trip][attr] = details[attr]
        trip_tottime_m2.append(details['triptime1_m'])
        if details['FC_tripd']==0: continue
        dist = details['FC_time1']/details['FC_tripd']
        Trips_time1_100_p1[trip]['tripLen_km'] = dist 
        trip_dist_km.append(dist)



KeyError: 'tripLen_km'

In [71]:
# output the completed trip dict to pickle
with  open('Trips_time1_100_github.pkl', 'wb') as o:
    
    pickle.dump(Trips_time1_100_p1,o)